In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:

# %%
from qdrant_client import QdrantClient
from datetime import datetime, timedelta
import pandas as pd

In [4]:

pd.set_option('display.max_colwidth', None)

# %% [markdown]
# ## Connect to Qdrant

# %%
def get_qdrant_client():
    """Get Qdrant client with default settings"""
    return QdrantClient(
        host="qdrant",  # Update if running in different environment
        port=6333
    )

# %% [markdown]
# ## Query Recent Chunks

# %%
def get_recent_chunks(collection_name="meeting_chunks", limit=100, hours_ago=24000*30):
    """
    Retrieve the most recent chunks from Qdrant.
    
    Args:
        collection_name (str): Name of the Qdrant collection
        limit (int): Maximum number of chunks to retrieve
        hours_ago (int): Look back period in hours
    
    Returns:
        pd.DataFrame: DataFrame containing chunk data
    """
    client = get_qdrant_client()
    
    # Calculate timestamp threshold
    time_threshold = (datetime.utcnow() - timedelta(hours=hours_ago)).isoformat()
    
    # Search with filtering by timestamp
    search_result = client.scroll(
        collection_name=collection_name,
        scroll_filter={"must": [
            {"key": "timestamp", "range": {"gte": time_threshold}}
        ]},
        limit=limit,
        with_payload=True,
        with_vectors=True
    )[0]  # scroll returns (points, next_page_offset)
    
    # Extract relevant fields from payload
    chunks_data = []
    for point in search_result:
        payload = point.payload
        chunks_data.append({
            'id': point.id,
            'timestamp': pd.to_datetime(payload.get('timestamp')),  # Convert to datetime
            'meeting_id': payload.get('meeting_id'),
            'content': payload.get('content'),
            'contextualized_content': payload.get('contextualized_content'),
            'chunk_index': payload.get('chunk_index'),
            'topic': payload.get('topic'),
            'speaker': payload.get('speaker'),
            'speakers': payload.get('speakers'),
            'vector_dim': len(point.vector) if point.vector else None
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(chunks_data)
    

    return df

In [5]:
get_recent_chunks()

,id,timestamp,meeting_id,content,contextualized_content,chunk_index,topic,speaker,speakers,vector_dim
0,032b931a-ce25-4c41-9225-3896f4b6723c,2024-06-17 11:07:08.001148+00:00,649abab5-c1ad-4066-83c7-cb0161aea310,Polar Wanderer: Да...,"The chunk occurs in a conversation among various mystical characters discussing the functionality of a system, with Polar Wanderer responding affirmatively to a previous statement, indicating engagement in the ongoing dialogue.",7,Acknowledgment,Polar Wanderer,"[Mystic Wizard, Polar Wanderer, Mystic Siren, Blazing Samurai, Mariner Comet]",1024
1,04066c2a-403a-43bf-8723-0eb08d2ba4ea,2024-06-17 11:51:54.943091+00:00,125a5015-5712-495d-84f9-1f54737e7ac9,"Polar Vulture: И релизнем первую версию, и уже можно разгуляться и сделать вообще шину между сервисами.","The document is a conversation between Mystic Wizard and Polar Vulture discussing technical issues related to meeting transcriptions and session management in a software system. The chunk reflects Polar Vulture's optimism about releasing the first version of their project and the potential for future enhancements, such as creating a service bus between services.",8,Future Development Plans,Polar Vulture,"[Polar Wanderer, Polar Vulture, Mystic Wizard, Harmonic Spirit]",1024
2,0424cb2e-44fd-4b56-baee-2efa70553ef5,2024-06-11 10:44:30.159113+00:00,48512bee-c060-4e74-8b05-ba19ea65ef9e,"Nova Alchemist: However, that said, rags have some limitations. When we do rag search, we are using some kind of semantic similarity and vectorizing everything in the rag. rack search we are using some kind of semantic similarity and vectorizing everything in the rack so here because we are vectorizing all the passages then we are losing all the connections and relationships that exist in the text because we are basically flattening the entire text and that exist in the text because we are basically flattening the entire text and represent it as a bunch of vectors. So we are not really encoding all of the semantic intent in the text. That's one of the problems with RAG in general or with any kind of vectorization. That's one of the problems with rag in general, or with any kind of vectorization. That's one of the problems with RAG in general, or with any kind of vectorization. So to put it in more plain English, this process of vectorization of text chunks So to put it in more plain English, this process of vectorization of text chunks is really just embedding, turning, let's say the whole sentence, one sentence in the chunk. is really just embedding, turning, let's say, the whole sentence, one sentence in the chunk. like one word in the sentence versus another word, not to mention their interrelationship is not, it's embedded in the embeddings, but not necessarily like a very tight, very, it doesn't pay, I don't know what's the right word to say. Explicitly, right? We are not explicitly describe the relationship between the words or the entities or the concept, right? The sentences. explicitly describe the relationship between the words or the entities or the concept, right? The sentences. So that's basically, so we're losing right here. We are not really including all of that information.","This chunk discusses the limitations of Retrieval Augmented Generation (RAG) systems, specifically focusing on the issues related to semantic similarity and vectorization, which result in the loss of connections and relationships within the text. It highlights how the process of embedding text into vectors fails to capture the explicit relationships between words, entities, and concepts, thereby compromising the semantic intent of the information.",5,Limitations of RAG Systems,Nova Alchemist,"[Terra Tsunami, Nova Alchemist, Eternal Swan, Amber Swan, Polar Paladin, Vigilant Prophet]",1024
3,05b47434-b5c9-4f24-82be-9cd3502e4d10,2024-06-17 11:51:54.943091+00:00,125a5015-5712-495d-84f9-1f54737e7ac9,"Polar Vulture: Хорошо, сейчас попробую. Насчет транскрипта вот здесь труднее. Почему? Е